In [1]:
import pickle

with open('titanic_step1_feature_change.pickle', 'rb') as pickle_filename:
    df = pickle.load(pickle_filename)

In [2]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,Initial,Family,Ticket_initial2,Ticket_Num_Cut
0,1,0.0,3,0,3.0,0.0,U,0,0.0,2,0,3
1,2,1.0,1,1,4.0,3.0,C,1,2.0,2,1,3
2,3,1.0,3,1,3.0,0.0,U,0,1.0,1,2,9
3,4,1.0,1,1,4.0,2.0,C,0,2.0,2,3,5
4,5,0.0,3,0,4.0,0.0,U,0,0.0,1,3,9


### Initial -> Survived 관계

In [17]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
# cf.go_offline(connected=True)

title_mapping = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Lady': 4, 'Don': 5, 'Rev': 6, 'Dr': 7,
                 'Major': 8, 'Sir': 9, 'Col': 10,  'Capt': 11, 'Jonkheer': 12}

ntrain = 891
train = df[:ntrain]
survived = train[train['Survived'] == 1]['Initial']
dead = train[train['Survived'] == 0]['Initial']
df_temp1 = pd.concat([survived, dead], axis=1, keys=['Survived', 'Dead'])
df_temp1.iplot(kind='histogram')

0, 1, 2, 3: Mr, Ms, Mrs, Master는 살아남았지만, 나머지 성을 가진 사람들은 모두 사망하였다.
특히 Mr 성(0번)-> 남성을 가진 사람은 대부분 사망하였다.

### Pclass -> Survived 관계

In [18]:
ntrain = 891
train = df[:ntrain]
survived = train[train['Survived'] == 1]['Pclass']
dead = train[train['Survived'] == 0]['Pclass']
df_temp1 = pd.concat([survived, dead], axis=1, keys=['Survived', 'Dead'])
df_temp1.iplot(kind='histogram')

Pclass: 1 (S)열에 탑승한 승객은 절반 정도 살아남은 반면, Q열과 R열에 탑승한 승객들은 살아남은 사람에 비해, 사망하는 사람이 훨씬 많다.

### Age -> Survived 관계

In [19]:
ntrain = 891
train = df[:ntrain]
survived = train[train['Survived'] == 1]['Age']
dead = train[train['Survived'] == 0]['Age']
df_temp1 = pd.concat([survived, dead], axis=1, keys=['Survived', 'Dead'])
df_temp1.iplot(kind='histogram')

영유아 ~ 20대는 살아남은 사람의 비율이 많은 반면, 나머지 연령대는 사망한 사람의 비율이 높다. 특히, 30~40대의 사망자의 비율이 매우 높은 편이다.

### Sex -> Survived 관계

In [20]:
ntrain = 891
train = df[:ntrain]
survived = train[train['Survived'] == 1]['Sex']
dead = train[train['Survived'] == 0]['Sex']
df_temp1 = pd.concat([survived, dead], axis=1, keys=['Survived', 'Dead'])
df_temp1.iplot(kind='histogram')

여성의 경우 생존한 사람의 비율이 남성에 비해 훨씬 높은 것을 알 수 있다.

- 등급이 높은 객실(S)등급이며, 연령대가 적고, 여성일 경우 살아남을 확률이 높다.
- 귀족 여성일 경우, 살아남을 확률이 높다.
- 1등급 객실의 여성은 살아남을 확률이 높다.

In [12]:
df['HighChance'] = 0
df.loc[(df['Pclass'] < 3) & (df['Age'] < 2) & (df['Sex'] == 1), 'HighChance'] = 6
df.loc[(df['Initial'] == 4), 'HighChance'] = 5
df.loc[(df['Pclass'] < 3) * (df['Age'] == 2) & (df['Sex'] == 1), 'HighChance'] = 4
df.loc[(df['Pclass'] == 1) & (df['Initial'] == 1), 'HighChance'] = 3
df.loc[(df['Pclass'] == 1) & (df['Initial'] == 2), 'HighChance'] = 2
df.loc[(df['Age'] == 0), 'HighChance'] = 1

In [21]:
ntrain = 891
train = df[:ntrain]
survived = train[train['Survived'] == 1]['HighChance']
dead = train[train['Survived'] == 0]['HighChance']
df_temp1 = pd.concat([survived, dead], axis=1, keys=['Survived', 'Dead'])
df_temp1.iplot(kind='histogram')


1: 영유아의 경우 생존자의 비율이 높은 것을 확인할 수 있다.
2: 귀족 여성이며(MRS), Pclass에 탑승한 경우 대부분 생존했다.
3: 귀족 남성이며(MR), Pclass에 탑승한 경우 대부분 생존했다.
4, 6: Pclass에 탑승했으며, 여성이며, 영유아 및 10대는 생존했지만 그 비율이 적은 편이다.

- 등급이 낮고, 청년 또는 노인인 남성일 경우 살아남을 확률이 낮다.

In [14]:
df['LowChance'] = 0
df.loc[(df['Sex'] == 0) & (df['Family'] == 1), 'LowChance'] = 1
df.loc[(df['Sex'] == 0) & (df['Family'] == 1) & (df['Pclass'] == 3), 'LowChance'] = 2

In [22]:
ntrain = 891
train = df[:ntrain]
survived = train[train['Survived'] == 1]['LowChance']
dead = train[train['Survived'] == 0]['LowChance']
df_temp1 = pd.concat([survived, dead], axis=1, keys=['Survived', 'Dead'])
df_temp1.iplot(kind='histogram')

1: 남성이며, 혼자왔을 경우 대부분 사망했다.
2: 남성이며, 혼자오고, 3등급 석에 앉은 경우 거의 사망했다.

In [16]:
with open('titanic_step2_feature_add.pickle', 'wb') as pickle_filename:
    pickle.dump(df, pickle_filename)